# Import packages

In [1]:
from ipywidgets import HTML
import ipywidgets as widgets
from ipyleaflet import Map, Polyline, Rectangle, basemaps, basemap_to_tiles, Polygon, FullScreenControl, Popup, WidgetControl
import pandas as pd
import numpy as np
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from ipyleaflet import Map, basemaps, basemap_to_tiles, Circle, FullScreenControl, LayerGroup
from ipywidgets.embed import embed_minimal_html
import sys
import scipy.stats
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from IPython.display import clear_output, display
sys.path.append('/datc/saab/notebooks')

# Define functions

In [2]:
def normalize(v):
    """
    Normalize a vector. Turn into a 1-module vector
    """
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm
def plot_directions_map(avg_directions, save=False, filename='default_filename'):
    m = Map(center = (22.205232, 114.123882), zoom = 12)#Define the map object
    m.layout.height = '100vh'
    i = 0
    for coords, direction in avg_directions.items():
        if direction[0] == 0 or direction[1] == 0:
            color_value = 'black'
            continue
        elif direction[0] > 0 and direction[1] > 0:
            color_value = 'green'
        elif direction[0] < 0 and direction[1] < 0:
            color_value = 'yellow'
        elif direction[0] < 0 and direction[1] > 0:
            color_value = 'red'
        elif direction[0] > 0 and direction[1] < 0:
            color_value = 'blue'
        line = Polyline(
            locations = [list(coords), list(coords+direction/1000)],
            color = color_value,
            fill_color= "transparent",
            weight = 3,
            opacity = 1)
        m.add_layer(line)
        i+=1
    m.add_control(FullScreenControl())
    if save:
        embed_minimal_html(filename, views=[m])
    return m

def Sort(sub_li, fieldnum): 
    """
    Sort a dataframe by a field(fieldnum)
    """
    return(sorted(sub_li, key = lambda x: x[fieldnum]))

# Import the data

In [3]:
# Code to import the data and remove NaN values from it
filename = '/datc/saab/reduced_area_clean.h5'
data = pd.read_hdf(filename, 'df')
#data = data[(50 < data.length )]
data = data.dropna()
data.head()
data.latitude = data['latitude'] + 47.72
data.longitude = data['longitude'] + 157.85
data


,mmsi,datetime,latitude,longitude,orientation,rateofturn,course,length,breadth,speed,vesseltype
171,56295,2018-11-30 16:00:00.707,22.199854,114.080508,98.0000,0.000000,89.00000,24.500000,3.099609,0.620117,0
189,0,2018-11-30 16:00:00.707,22.216311,114.091476,166.2500,0.000000,165.00000,54.812500,13.703125,3.759766,0
190,0,2018-11-30 16:00:00.707,22.191633,114.090895,275.2500,0.000000,136.37500,148.750000,19.203125,0.040009,0
191,0,2018-11-30 16:00:00.707,22.167373,114.086139,41.8125,0.000000,42.40625,57.906250,8.000000,4.019531,0
205,93,2018-11-30 16:00:00.707,22.259018,114.105921,144.7500,-3.400391,167.00000,21.296875,2.699219,0.560059,0
...,...,...,...,...,...,...,...,...,...,...,...
42501846,54874,2018-12-01 15:41:15.706,22.248155,114.141039,176.7500,0.000000,360.00000,41.000000,18.000000,0.000000,0
42501852,11655,2018-12-01 15:41:15.706,22.247677,114.148988,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0
42501855,11797,2018-12-01 15:41:15.706,22.244843,114.125698,122.8125,3.400391,81.50000,24.593750,3.400391,4.589844,0
42501861,54044,2018-12-01 15:41:15.706,22.250201,114.131207,145.3750,0.000000,146.37500,14.000000,4.000000,8.601562,0


In [4]:
data.describe()

,mmsi,latitude,longitude,orientation,rateofturn,course,length,breadth,speed,vesseltype
count,1.905895e+06,1.905895e+06,1.905895e+06,1905895.00,1905895.00,1905895.000,1.905895e+06,1.905895e+06,1.905895e+06,1.905895e+06
mean,1.456896e+04,2.221842e+01,1.141437e+02,NaN,NaN,NaN,NaN,NaN,NaN,5.116756e-02
std,2.205727e+04,3.140755e-02,4.314830e-02,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,6.377481e-01
min,0.000000e+00,2.215582e+01,1.140757e+02,0.00,-337.00,0.000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.219328e+01,1.141069e+02,108.50,0.00,92.875,1.750000e+01,2.500000e+00,6.098633e-01,0.000000e+00
50%,0.000000e+00,2.222191e+01,1.141355e+02,162.50,0.00,170.375,2.690625e+01,4.898438e+00,3.140625e+00,0.000000e+00
75%,3.119100e+04,2.224460e+01,1.141775e+02,298.25,0.00,281.750,4.618750e+01,1.070312e+01,4.371094e+00,0.000000e+00
max,6.506700e+04,2.227465e+01,1.142476e+02,360.00,405.75,360.000,5.000000e+02,1.917500e+02,1.288750e+02,8.000000e+00


# Get a array containing all ship's sequences --> ships_info

In [5]:
CRAFT_ID_list = data.mmsi.unique()#Get the mmsi unique values into a list:
CRAFT_ID_list = CRAFT_ID_list[CRAFT_ID_list!=0]
ships_info = []
ship_number = 0
for rowid in CRAFT_ID_list:
    #Start with empty lists
    npinfo, infolist = [], []
    #Get a numpy array composed by 'latitude', 'longitude', 'orientation', 'length', 'breadth'
    npinfo = data[data.mmsi == rowid][['latitude', 'longitude', 'speed', 'length', 'datetime']].values
    
    ships_info.append(npinfo)
    
    ship_number+=1
    if ship_number%100 == 0:
        print(ship_number, '/', len(CRAFT_ID_list))
        clear_output(wait=True)
print('finished')
ships_info = [Sort(row, -1) for row in ships_info]

finished


# Split data into train/tests sets

In [6]:
border = int(0.9*len(ships_info))
train_ships = ships_info[:border]
test_ships = ships_info[border:]

In [7]:
CRAFT_ID_list[border:]

array([45947, 32691, 54214, 55180,  5621,  2403,  5691, 41568, 13179,
       59333, 60604, 29628, 36923, 54034, 11666,  2931, 57637,  7419,
       59531, 34821,  2921, 12972,  1089, 40219, 54042, 39739,  4294,
        6953,  8295, 35739, 51677,  3865, 14875, 15999, 29579,  1882,
       37752, 11276, 52797, 65067, 16875, 38923, 54114, 11797, 60805,
       60806, 53963,  4091, 33655, 33698, 13459], dtype=uint64)

# Get every ship direction

In [8]:
directions = []
for ship in train_ships:
    directions.append([])
    for i in range(len(ship)-1):
        try:
            #direc = np.array([ship[i][2],ship[i][3]])
            direc = normalize(ship[i+1][:2]-ship[i][:2])
            directions[-1].append(direc)
        except Exception as e:
            print('problem!!!!!!!!!!!!!!', e)
            pass

# Get the bounds of the grid according to a number of decimals defined

In [9]:
decimals = 3
radius = 10**(-decimals)
data = np.round(data[['latitude', 'longitude', 'speed']].values, decimals)


minimum_lat = min(data[:,0])
maximum_lat = max(data[:,0])
minimum_lon = min(data[:,1])
maximum_lon = max(data[:,1])
print(minimum_lat, maximum_lat, minimum_lon, maximum_lon)

22.156 22.275 114.076 114.248


# Define the possible values of lat-long along the grid

In [10]:
lon_coordinates = np.arange(start=minimum_lon, stop=maximum_lon, step=np.round(10**(-decimals), decimals))
lat_coordinates = np.arange(start=minimum_lat, stop=maximum_lat, step=np.round(10**(-decimals), decimals))

In [11]:
print('grid size:', len(lon_coordinates), 'x', len(lat_coordinates))

grid size: 173 x 119


# Get all node's directions

In [12]:
directions_list = {}
i=0
for lat in lat_coordinates:
    for lon in lon_coordinates:            
        directions_list[(np.round(lat,decimals), np.round(lon,decimals))]=[]
        i+=1

for i in range(len(train_ships)):
    print('Computing row : ', i+1)
    clear_output(wait=True)
    for j in range(len(train_ships[i])-1):
        try:
            directions_list[(np.round(train_ships[i][j][0],decimals), np.round(train_ships[i][j][1],decimals))].append(directions[i][j])
        except:
            directions_list[(np.round(train_ships[i][j][0],decimals), np.round(train_ships[i][j][1],decimals))] = [directions[i][j]]
print('Computed ', len(lon_coordinates)*len(lat_coordinates), ' nodes.')

Computed  20587  nodes.


# Get the average direction for each node

In [13]:
avg_directions = {}
for coords, directions in directions_list.items():
    if len(directions) != 0:
        avg_directions[coords] = np.nanmean(np.array(directions), axis=0)
    else:
        avg_directions[coords] = np.array([0, 0])

In [35]:
plot_directions_map(avg_directions, save=True, filename='directionsmap.html')

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# Get the mean and std deviation for each directions node (Normal distribution)

In [14]:
norm_dist_params = {}
for coords, directions in directions_list.items():
    if len(directions) != 0:
        norm_dist_params[coords] = [[np.mean(np.array(directions)[:,0]), np.std(np.array(directions)[:,0])], [np.mean(np.array(directions)[:,1]), np.std(np.array(directions)[:,1])]]
    else:
        norm_dist_params[coords] = [[np.nan, np.nan],[np.nan, np.nan]]

# Aproximate the distribution function of each node using KDE method

In [15]:
KDE_density_func = {}

class nan_obj:
    def score_samples(x,y):
         return [np.nan]
for coords, directions in directions_list.items():
    if len(directions) > 20:
        KDE_density_func[coords] = KernelDensity(kernel='gaussian', bandwidth=0.001).fit(directions)
    else:
        KDE_density_func[coords] = nan_obj()

# Explore every node with some plots

In [16]:
keys = [x[0] for x in sorted(directions_list.items(), key=lambda x: len(x[1]), reverse=True) if len(x[1]) > 0]
max_steps = len(keys)
bandwidth = 0.2
# Define the slider
ships_slider = widgets.IntSlider(
    value=3500,
    min=0,
    max=max_steps,
    step=1,
    description='Ships: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Plot it

widget_steps = WidgetControl(widget=ships_slider)
previous_value = 0
import seaborn as sns
#Define the update function for the slider
def update_plot(ships_slider):
    coords = keys[ships_slider]
    directions = directions_list[coords]
    mean0 = norm_dist_params[coords][0][0]
    std0 = norm_dist_params[coords][0][1]
    mean1 = norm_dist_params[coords][1][0]
    std1 = norm_dist_params[coords][1][1]
    print('mean LAT: ', mean0,'\nstd  LAT: ', std0)
    print('mean LON: ', mean0,'\nstd  LON: ', std0)
    #Define a bound for the plot
    x = np.linspace(mean0 - 2*abs(std0), mean0 + 2*abs(std0),100)
    y = scipy.stats.norm(mean0,std0).pdf(x)
    
    #Normal distribution aproximation
    plt.plot(x,y,c='blue', alpha=0.5)
    
    x = np.linspace(mean1 - 2*abs(std1), mean1 + 2*abs(std1),100)
    y = scipy.stats.norm(mean1,std1).pdf(x)
    #Normal distribution aproximation
    plt.plot(x,y,c='red', alpha=0.5)
    plt.hist(np.array(directions)[:,0], bins=100,density=True, color='darkblue', alpha=0.5)
    plt.hist(np.array(directions)[:,1], bins=100,density=True, color='darkred', alpha=0.5)
    plt.title('Normal Distribution Aproximation')
    plt.xlabel("LAT/LON of the node's directions")
    plt.legend(['Normal aproximation to LON distribution',
                'Normal aproximation to LAT actual distribution',
                'Actual LON distribution',
                'Actual LAT distribution'])
    plt.ylabel('Frequency')
    plt.show()
    # Actual data
    plt.hist(np.array(directions)[:,0], bins=100, normed=True, color='darkblue', alpha=0.5)
    plt.hist(np.array(directions)[:,1], bins=100, normed=True, color='darkred', alpha=0.5)
    sns.kdeplot(np.array(directions)[:,0], kernel = 'tri', color='darkblue', bw=bandwidth)
    sns.kdeplot(np.array(directions)[:,1], kernel='tri',  color='darkred', bw=bandwidth)
    plt.legend(['LON actual distribution',
                'LAT actual distribution',
                'LON KDE infered distribution',
                'LAT KDE infered distribution'])
    plt.title('KDE Distribution Aproximation')
    plt.xlabel("LAT/LON of the node's directions")
    plt.ylabel('Frequency')
    plt.show()
    
    ax = sns.kdeplot(np.array(directions)[:,0], np.array(directions)[:,1], cbar=True, shade=True)
    plt.show()
    f, ax = plt.subplots(figsize=(6, 6))
    sns.kdeplot(np.array(directions)[:,0], np.array(directions)[:,1], ax=ax, bw = bandwidth)
    sns.rugplot(np.array(directions)[:,0], color="g", ax=ax)
    sns.rugplot(np.array(directions)[:,1], vertical=True, ax=ax);
    plt.title('Gaussian KDE Distribution Aproximation')
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
    if len(directions)>0:
        print('POINT ('+str(coords[0])+', '+ str(coords[1])+ ')')
        plt.quiver([0]*len(directions),[0]*len(directions), list(np.array(directions)[:,0]), list(np.array(directions)[:,1]), alpha=0.1)
    else:
        print('POINT ('+str(coords[0])+', '+ str(coords[1])+ ')')
        print('NO INFO')
    plt.show()
    if len(directions)>0:
        print('POINT ('+str(coords[0])+', '+ str(coords[1])+ ')')
        plt.quiver([0]*len(directions),[0]*len(directions), list(np.array(directions)[:,0]), list(np.array(directions)[:,1]), alpha=0.1)
    else:
        print('POINT ('+str(coords[0])+', '+ str(coords[1])+ ')')
        print('NO INFO')
    plt.show()
widgets.interactive(update_plot, ships_slider=ships_slider)

interactive(children=(IntSlider(value=3500, continuous_update=False, description='Ships: ', max=10348), Output…

# Define the buffer class to reduce noise

In [17]:
from collections import Counter

class ship_buffer:
    def __init__(self, threshold):
        self.items = []
        self.threshold = threshold

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):
        self.items.insert(0,item)

    def dequeue(self):
        return self.items.pop()

    def size(self):
        return len(self.items)
    
    def color(self):
        if sum(elem[1] < .4  for elem in self.items) >= 0.5*len(self.items):
            return 'blue'#Anchored
        elif sum(np.isnan(elem[0]) for elem in self.items) >= 0.5*len(self.items):
            return 'orange'#Too many nan values
        elif sum(elem[0] < self.threshold for elem in self.items) >= len(self.items):
            return 'red'#Anomaly
        else:
            return 'green'#OK
    
    def print_queue(self):
        #print(self.items)
        return

# Threshold and queue size

In [18]:
th = 0.5
queue_size = 10

# Anomalies Map

In [19]:
from scipy import stats
import math
from ipywidgets import Layout
m = Map(center = (22.205232, 114.123882), zoom = 12)#Define the map object

#To define  the maximum number of steps we will be able to take with the slider
max_steps = len(test_ships)
ships_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=max_steps,
    step=1,
    description='Ships: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
"""out1 = widgets.Output(layout=Layout(width='100%', height='100%'))
with out1:
    plt.figure(figsize=(2,2))
    arrows = plt.quiver(0,0, 1, 1, color='blue')
    new_dir = plt.quiver(0,0, -1, -1, color='red')
    plt.show()
tab = widgets.Tab(children = [out1])
tab.set_title(0, 'First')"""

"""widget_control1 = WidgetControl(widget=tab, position='bottomright')
m.add_control(widget_control1)"""
prob_env = ship_buffer(th)
for i in range(queue_size):
    prob_env.enqueue([np.nan, np.nan])

widget_steps = WidgetControl(widget=ships_slider, position='topright')
m.add_control(widget_steps)
m.add_control(FullScreenControl())
dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
m.add_layer(dark_matter_layer)
for ship in train_ships:
    line = Polyline(
        locations = [[list(elem[:2]) for elem in ship]],
        color = 'gray',
        fill_color= "transparent",
        weight = 1,
        opacity = 0.1)
    m.add_layer(line)
previous_value = 0


def update_map(ships_slider):
    global previous_value, m
    if previous_value > ships_slider:
        ini, end = 0, ships_slider
    else:
        ini, end = previous_value, ships_slider
        
    step = 1
    for i in range(ini, end, step):
        try:
            color_value = 'green'
            for j in range(0,len(test_ships[i])-step, step):
                #slope = np.array([test_ships[i][j][2], test_ships[i][j][3]])
                slope = normalize((test_ships[i][j+step][:2] - test_ships[i][j][:2])/step)
                dist_value = (KDE_density_func[(np.round(test_ships[i][j][0], decimals), np.round(test_ships[i][j][1], decimals))].score_samples([slope])[0])
                #dist_value = 10**(KDE_density_func[(np.round(test_ships[i][j][0], decimals), np.round(test_ships[i][j][1], decimals))].score_samples([slope])[0])
                #print(dist_value)
                prob_env.dequeue()
                prob_env.enqueue([dist_value, test_ships[i][j][2], len(directions_list[(np.round(test_ships[i][j][0], decimals), np.round(test_ships[i][j][1], decimals))])])
                color_value = prob_env.color()
                

                directions = directions_list[(np.round(test_ships[i][j][0], decimals), np.round(test_ships[i][j][1],decimals))]
                """with out1:
                    for artist in plt.gca().lines + plt.gca().collections:
                        artist.remove()
                    if len(directions)>0:
                        print('POINT ('+str(coords[0])+', '+ str(coords[1])+ ')')
                        arrows = plt.quiver([0]*len(directions),[0]*len(directions), list(np.array(directions)[:,1]), list(np.array(directions)[:,0]), alpha=0.1)
                    else:
                        print('POINT ('+str(coords[0])+', '+ str(coords[1])+ ')')
                        print('NO INFO')
                    actual_dir = normalize((test_ships[i][j+step][:2] - test_ships[i][j][:2]))
                    new_dir = plt.quiver(0,0, actual_dir[1], actual_dir[0], color=color_value)
                    plt.show() """               
                
                line = Polyline(
                    locations = [list(test_ships[i][j][:2]), list(test_ships[i][j+step][:2])],
                    color = color_value,
                    fill_color= "transparent",
                    weight = 2,
                    opacity = 1)
                m.add_layer(line)
        except Exception as e:
            print(e)
            pass

    previous_value = ships_slider
display(m)
widgets.interactive(update_map, ships_slider=ships_slider)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

interactive(children=(IntSlider(value=0, continuous_update=False, description='Ships: ', max=51), Output()), _…